#### Cryptography
#### Task № 8 - HASH WITH DES (Davies–Meyer)
#### Ivan Rybin - ITMO JB SE MA 2021

In [44]:
import des_data

In [45]:
def xor_bytes(l, r):
    return bytearray([lb ^ rb for lb, rb in zip(l, r)])

def xor_bits(l, r):
    return "".join([str(int(lb) ^ int(rb)) for lb, rb in zip(l, r)])

def stobytes(s):
    return s.encode('utf-8')

def sto64(bits):
    while len(bits) % 64 != 0:
        bits += '0'
    return bits

def to8bits(bits):
    while len(bits) < 8:
        bits = '0' + bits
    return bits

def to4bits(bits):
    while len(bits) < 4:
        bits = '0' + bits
    return bits

def stobits(s):
    b = ""
    for c in stobytes(s):
        b += to8bits(bin(c)[2:])
    return b

def btoi(b):
    i = 0
    k = 0
    for c in reversed(b):
        i += int(c) * (2 ** k)
        k += 1
    return i

def bytestos(b):
    return b.decode('utf-8')

def bitstos(bits):
    s = []
    for b in split_to_blocks(bits, 8):
        s.append(btoi(b))
    return bytearray(s)

def perm(p, b):
    return ''.join([b[i] for i in p])

def split_to_blocks(b, block_size):
    blocks = []
    for i in range(1, int((len(b) + block_size) / block_size)):
        blocks.append(b[(i - 1) * block_size: i * block_size])
    return blocks

In [46]:
def ip_perm(b):
    return perm(des_data.ip_perm, b)

def ip_inv_perm(b):
    return perm(des_data.ip_inv_perm, b)

def p_perm(b):
    return perm(des_data.p_perm, b)

def g_perm(b):
    return perm(des_data.g_perm, b)

def h_perm(b):
    return perm(des_data.h_perm, b)

In [47]:
# Encrypted function f(R, K)
def extend_perm(b):
    return perm(des_data.extend_perm, b)


def get_s_block_value(block, s_block):
    row = btoi(block[0] + block[-1]) # b1b6
    col = btoi(block[1:-1]) # b2b3b4b5
    return s_block[row * 16 + col]


def s_blocks_func(blocks):
    new_bits = ''
    for i in range(0, 8):
        s_block = des_data.s_blocks[i]
        new_block = get_s_block_value(blocks[i], s_block)
        new_bits += to4bits(bin(new_block)[2:])
    return new_bits

# R - 6 bit b1b2b3b4b5b6

# R(i-1) -> E -> r ^ key -> s1-s8 -> 8 blocks x 4 bites -> P -> f(R(i-1), K(i))
def encrypt_func(r, key):
    # r: 32 bites -> r_extend: 48 bites
    r_extend = extend_perm(r)    
    r_xor_key = xor_bits(r_extend, key) # key: 48 bites
    blocks = split_to_blocks(r_xor_key, 6) # 8 blocks x 6 bites
    new_r = s_blocks_func(blocks)
    new_r = p_perm(new_r)
    return new_r

In [48]:
# Key generator
def key64_56(key):
    return g_perm(key)


def cyclic_offset(b, offset):
    new_b = ''
    for i in range(offset, offset + len(b)):
        new_b += b[i % len(b)]
    return new_b


def get_all_keys(key):
    c = []
    d = []
    keys = []
    key_56 = key64_56(key)
    c.append(key_56[0:28])
    d.append(key_56[28:])
    for i in range(1, 17): # from 1 to 16
        offset = des_data.key_offsets[i - 1]
        c_i = cyclic_offset(c[i - 1], offset)
        d_i = cyclic_offset(d[i - 1], offset)
        key_i = h_perm(c_i + d_i)
        keys.append(key_i)
        c.append(c_i); d.append(d_i)
        
    return keys

In [58]:
# DES algotithm
def DES_round(block, key):
    l = block[0:32]
    r = block[32:]
    new_l = r; new_r = xor_bits(l, encrypt_func(r, key))
    return new_l, new_r

def DES_block(block, key):
    keys = get_all_keys(key) # all keys k_i, i=0..16
    # ip permutation
    block = ip_perm(block)
    # 16 rounds
    for i in range(0, 16):
        key = keys[i]
        # encryption function f(R, K)
        l, r = DES_round(block, key)
        block = l + r
    # LR-permutation 
    block = block[32:] + block[0:32]
    # ip inv perm
    block = ip_inv_perm(block)
        
    return block

# Davies–Meyer
def DES_HASH(s, init_key):
    bits = sto64(stobits(s))
    blocks = split_to_blocks(bits, 64) # N blocks x 64 bits
    H = sto64(stobits(init_key))
    for block in blocks:
        H = DES_block(block=H, key=block) # H_i = E(m_i, H_i-1) xor H_i-1, where - m_i = key = open
    return H

In [59]:
init_key = 'crypto42'
message = 'this message was encrypted with DES algorithm with key "crypto42"'
message_bits = sto64(stobits(message))
print(f'Source message ({len(message_bits)} bits):\n\n{message_bits}')

Source message (576 bits):

011101000110100001101001011100110010000001101101011001010111001101110011011000010110011101100101001000000111011101100001011100110010000001100101011011100110001101110010011110010111000001110100011001010110010000100000011101110110100101110100011010000010000001000100010001010101001100100000011000010110110001100111011011110111001001101001011101000110100001101101001000000111011101101001011101000110100000100000011010110110010101111001001000000010001001100011011100100111100101110000011101000110111100110100001100100010001000000000000000000000000000000000000000000000000000000000


In [60]:
H = DES_HASH(message, init_key)
print(f"DES HASH LEN :{len(H)}")
print(f"DES HASH     :{H}")

DES HASH LEN :64
DES HASH     :1010000000001010001010101110110010010111111110101111111100011111
